<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [13]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [14]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [15]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [16]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [17]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [18]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [19]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [20]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [21]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 823ms/step - loss: 0.6709 - accuracy: 0.5900 - val_loss: 0.6333 - val_accuracy: 0.6829
Epoch 2/10
26/26 [==============================] - 21s 807ms/step - loss: 0.6262 - accuracy: 0.6411 - val_loss: 0.5913 - val_accuracy: 0.7268
Epoch 3/10
26/26 [==============================] - 22s 847ms/step - loss: 0.5627 - accuracy: 0.7251 - val_loss: 0.5546 - val_accuracy: 0.7659
Epoch 4/10
26/26 [==============================] - 21s 813ms/step - loss: 0.5114 - accuracy: 0.7664 - val_loss: 0.5744 - val_accuracy: 0.6732
Epoch 5/10
26/26 [==============================] - 20s 780ms/step - loss: 0.4783 - accuracy: 0.7786 - val_loss: 0.4613 - val_accuracy: 0.8098
Epoch 6/10
26/26 [==============================] - 24s 911ms/step - loss: 0.4458 - accuracy: 0.7993 - val_loss: 0.4184 - val_accuracy: 0.8293
Epoch 7/10
26/26 [==============================] - 20s 759ms/step - loss: 0.3944 - accuracy: 0.8382 - val_loss: 0.3423 - val_accuracy: 0.8683

In [ ]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [22]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.69.h5
26/26 - 22s - loss: 0.6830 - accuracy: 0.5584 - val_loss: 0.6873 - val_accuracy: 0.4390
Epoch 2/5

Epoch 00002: saving model to weights.02-0.68.h5
26/26 - 21s - loss: 0.6600 - accuracy: 0.6314 - val_loss: 0.6816 - val_accuracy: 0.4537
Epoch 3/5

Epoch 00003: saving model to weights.03-0.61.h5
26/26 - 20s - loss: 0.6298 - accuracy: 0.6971 - val_loss: 0.6129 - val_accuracy: 0.7561
Epoch 4/5

Epoch 00004: saving model to weights.04-0.57.h5
26/26 - 20s - loss: 0.5759 - accuracy: 0.7640 - val_loss: 0.5650 - val_accuracy: 0.7561
Epoch 5/5

Epoch 00005: saving model to weights.05-0.48.h5
26/26 - 21s - loss: 0.5090 - accuracy: 0.7652 - val_loss: 0.4798 - val_accuracy: 0.8146


In [23]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 25s - loss: 0.6773 - accuracy: 0.5949 - val_loss: 0.6321 - val_accuracy: 0.7707


In [24]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6835 - accuracy: 0.5596 - val_loss: 0.6410 - val_accuracy: 0.7415
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6288 - accuracy: 0.6740 - val_loss: 0.6263 - val_accuracy: 0.6049


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [25]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 28s - loss: 0.6733 - accuracy: 0.5669 - val_loss: 0.6590 - val_accuracy: 0.6098
Epoch 2/50
26/26 - 20s - loss: 0.6337 - accuracy: 0.6655 - val_loss: 0.6181 - val_accuracy: 0.6927
Epoch 3/50
26/26 - 20s - loss: 0.5736 - accuracy: 0.7409 - val_loss: 0.5594 - val_accuracy: 0.7463
Epoch 4/50
26/26 - 20s - loss: 0.5367 - accuracy: 0.7409 - val_loss: 0.4795 - val_accuracy: 0.8341
Epoch 5/50
26/26 - 19s - loss: 0.4860 - accuracy: 0.7701 - val_loss: 0.4301 - val_accuracy: 0.8878
Epoch 6/50
26/26 - 20s - loss: 0.4097 - accuracy: 0.8200 - val_loss: 0.4479 - val_accuracy: 0.8146
Epoch 7/50
26/26 - 19s - loss: 0.3595 - accuracy: 0.8625 - val_loss: 0.2871 - val_accuracy: 0.9268
Epoch 8/50
26/26 - 20s - loss: 0.2944 - accuracy: 0.8966 - val_loss: 0.2328 - val_accuracy: 0.9268
Epoch 9/50
26/26 - 20s - loss: 0.2867 - accuracy: 0.8881 - val_loss: 0.1889 - val_accuracy: 0.9561
Epoch 10/50
26/26 - 20s - loss: 0.2093 - accuracy: 0.9282 - val_loss: 0.1458 - val_accuracy: 0.9659
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [26]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 811ms/step - loss: 0.6614 - accuracy: 0.6058 - val_loss: 0.6295 - val_accuracy: 0.7366
Epoch 2/5
26/26 [==============================] - 19s 748ms/step - loss: 0.5880 - accuracy: 0.7226 - val_loss: 0.5533 - val_accuracy: 0.7805
Epoch 3/5
26/26 [==============================] - 19s 743ms/step - loss: 0.5188 - accuracy: 0.7652 - val_loss: 0.5926 - val_accuracy: 0.6293
Epoch 4/5
26/26 [==============================] - 19s 734ms/step - loss: 0.4569 - accuracy: 0.8041 - val_loss: 0.3884 - val_accuracy: 0.8390
Epoch 5/5
26/26 [==============================] - 20s 767ms/step - loss: 0.3996 - accuracy: 0.8358 - val_loss: 0.3364 - val_accuracy: 0.8732


In [27]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.605839,0.661793,0.736585,0.629456
1,1,0.722628,0.588885,0.780488,0.553266
2,2,0.765207,0.516392,0.629268,0.592605
3,3,0.804136,0.457945,0.839024,0.388378
4,4,0.835766,0.400521,0.873171,0.336426


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [28]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 21s 807ms/step - loss: 0.6850 - accuracy: 0.5341 - val_loss: 0.6799 - val_accuracy: 0.6976

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 19s 745ms/step - loss: 0.6716 - accuracy: 0.6241 - val_loss: 0.6840 - val_accuracy: 0.4537

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 19s 735ms/step - loss: 0.6659 - accuracy: 0.5499 - val_loss: 0.6800 - val_accuracy: 0.4585

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 757ms/step - loss: 0.6622 - accuracy: 0.5560 - val_loss: 0.6778 - val_accuracy: 0.4732

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 769ms/step - loss: 0.6

In [ ]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [29]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 20s 780ms/step - loss: 0.6818 - accuracy: 0.5414 - val_loss: 0.6895 - val_accuracy: 0.4341
Epoch 2/50
26/26 [==============================] - 20s 765ms/step - loss: 0.6659 - accuracy: 0.5499 - val_loss: 0.6734 - val_accuracy: 0.4878
Epoch 3/50
26/26 [==============================] - 20s 765ms/step - loss: 0.6472 - accuracy: 0.6460 - val_loss: 0.6544 - val_accuracy: 0.5707
Epoch 4/50
26/26 [==============================] - 19s 735ms/step - loss: 0.6196 - accuracy: 0.7372 - val_loss: 0.6189 - val_accuracy: 0.6878
Epoch 5/50
26/26 [==============================] - 20s 777ms/step - loss: 0.5790 - accuracy: 0.7798 - val_loss: 0.5800 - val_accuracy: 0.7024
Epoch 6/50
26/26 [==============================] - 19s 746ms/step - loss: 0.5173 - accuracy: 0.8200 - val_loss: 0.4938 - val_accuracy: 0.8293
Epoch 7/50
26/26 [==============================] - 20s 771ms/step - loss: 0.4275 - accuracy: 0.8759 - val_loss: 0.4421 - val_accuracy: 0.7902

In [ ]:
%tensorboard --logdir log_dir